# OPTICAL MARK RECOGNITION (OMR) 
### MCQ Automated Grading

In [28]:
import cv2
import numpy as np
from ipynb.fs.full.utils import stackImages
from ipynb.fs.full.utils import rectContour
from ipynb.fs.full.utils import getCornerPoints
from ipynb.fs.full.utils import reorder
from ipynb.fs.full.utils import splitBoxes
from ipynb.fs.full.utils import drawGrid
from ipynb.fs.full.utils import showAnswers

In [29]:
path = '1.jpg'
widthImg = 600
heightImg = 600
correctAnswers = [1,2,0,1,4]

In [33]:
img = cv2.imread(path)

# PREPROCESSING
img = cv2.resize(img,(widthImg,heightImg))
imgContours = img.copy()
imgBiggestContours = img.copy()
imgFinal = img.copy()
imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
imgCanny = cv2.Canny(imgBlur,10,50)

# FIND CONTOURS
contours,hierarchy = cv2.findContours(imgCanny,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
cv2.drawContours(imgContours,contours,-1,(0,255,0),10)

# FIND RECTANGLES
rectCon = rectContour(contours)
biggestContour = getCornerPoints(rectCon[0])
gradePoints = getCornerPoints(rectCon[1])


# DRAW CONTOURS
if biggestContour.size != 0 and gradePoints.size != 0:
    cv2.drawContours(imgBiggestContours,biggestContour,-1,(0,255,0),20)
    cv2.drawContours(imgBiggestContours,gradePoints,-1,(255,0,0),20)

    # REORDER
    biggestContour = reorder(biggestContour)
    gradePoints = reorder(gradePoints)
    
    # SEPARATING ANSWERS SECTION
    pt1 = np.float32(biggestContour)
    pt2 = np.float32([[0,0],[widthImg,0],[0,heightImg],[widthImg,heightImg]])
    matrix = cv2.getPerspectiveTransform(pt1,pt2)
    imgWrapColored = cv2.warpPerspective(img,matrix,(widthImg,heightImg))
    
    # SEPARATING GRADING SECTION
    ptG1 = np.float32(gradePoints)
    ptG2 = np.float32([[0,0],[325,0],[0,150],[325,150]])
    matrixG = cv2.getPerspectiveTransform(ptG1,ptG2)
    imgGradeDisplay = cv2.warpPerspective(img,matrixG,(325,150))
    
    # LOGIC => 
    # Marks Bubbles = More Pixels
    # Unmark Bubbles = Less Pixels
    
    # APPLY THRESHOLD
    imgWrapGray = cv2.cvtColor(imgWrapColored,cv2.COLOR_BGR2GRAY)
    imgThresh = cv2.threshold(imgWrapGray,170,255,cv2.THRESH_BINARY_INV)[1]
    
    # SEPARATING ALL BUBBLES
    boxes = splitBoxes(imgThresh)
    
    # CALCULATING PIXELS
    myPixelVal = np.zeros((5,5)) # (question,choices)
    countCol = 0
    countRow = 0
    
    for box in boxes:
        totalPixels = cv2.countNonZero(box)
        myPixelVal[countRow][countCol] = totalPixels
        countCol = countCol + 1
        
        if countCol == 5:
            countRow = countRow + 1
            countCol = 0
    
    # INDEX VALUES OF MARKING
    markingIndex = []
    
    for x in range(0,5):
        arr = myPixelVal[x]
        indexVal = np.where(arr == np.amax(arr))
        markingIndex.append(indexVal[0][0])
    
    
    # GRADING
    grading = []
    
    for x in range(0,5):
        if correctAnswers[x] == markingIndex[x]:
            grading.append(1)
        else:
            grading.append(0)
            
    score = (sum(grading) / 5) * 100
    
    # DISPLAYING ANSWERS
    imgRawDrawing = np.zeros_like(imgWrapColored)
    imgRawDrawing = showAnswers(imgRawDrawing,markingIndex,grading,correctAnswers)
    
    invMatrix = cv2.getPerspectiveTransform(pt2,pt1)
    imgInvWarp = cv2.warpPerspective(imgRawDrawing,invMatrix,(widthImg,heightImg))
    
    imgFinal = cv2.addWeighted(imgFinal,1,imgInvWarp,1,0)
    
    # DISPLAYING GRADE SCORE
    imgRawGrade = np.zeros_like(imgGradeDisplay)
    cv2.putText(imgRawGrade,str(int(score))+'%',(60,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,255),3)
    
    invMatrixG = cv2.getPerspectiveTransform(ptG2,ptG1)
    imgInvGradeDisplay = cv2.warpPerspective(imgRawGrade,invMatrixG,(widthImg,heightImg))
    
    imgFinal = cv2.addWeighted(imgFinal,1,imgInvGradeDisplay,1,0)
    
imgBlank = np.zeros_like(img)    
imgArray = ([img,imgGray,imgCanny,imgContours],
           [imgBiggestContours,imgWrapColored,imgThresh,imgFinal])
imgStacked = stackImages(imgArray,0.5)

resultStacked = stackImages(([img,imgFinal]),0.8)

# cv2.imshow('Final Results',imgFinal)
cv2.imshow('Stacked Images',imgStacked)
cv2.imshow('ORM',resultStacked)
cv2.waitKey(0)    
cv2.destroyAllWindows()

In [34]:
cv2.imwrite('orm.jpg',resultStacked)
cv2.imwrite('img_stacked.jpg',imgStacked)

True

## Webcam

## ORM Function

In [24]:
def orm(img):

    try:
        # PREPROCESSING
        img = cv2.resize(img,(widthImg,heightImg))
        imgContours = img.copy()
        imgBiggestContours = img.copy()
        imgFinal = img.copy()
        imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
        imgCanny = cv2.Canny(imgBlur,10,50)

        # FIND CONTOURS
        contours,hierarchy = cv2.findContours(imgCanny,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
        cv2.drawContours(imgContours,contours,-1,(0,255,0),10)

        # FIND RECTANGLES
        rectCon = rectContour(contours)
        biggestContour = getCornerPoints(rectCon[0])
        gradePoints = getCornerPoints(rectCon[1])


        # DRAW CONTOURS
        if biggestContour.size != 0 and gradePoints.size != 0:
            cv2.drawContours(imgBiggestContours,biggestContour,-1,(0,255,0),20)
            cv2.drawContours(imgBiggestContours,gradePoints,-1,(255,0,0),20)

            # REORDER
            biggestContour = reorder(biggestContour)
            gradePoints = reorder(gradePoints)

            # SEPARATING ANSWERS SECTION
            pt1 = np.float32(biggestContour)
            pt2 = np.float32([[0,0],[widthImg,0],[0,heightImg],[widthImg,heightImg]])
            matrix = cv2.getPerspectiveTransform(pt1,pt2)
            imgWrapColored = cv2.warpPerspective(img,matrix,(widthImg,heightImg))

            # SEPARATING GRADING SECTION
            ptG1 = np.float32(gradePoints)
            ptG2 = np.float32([[0,0],[325,0],[0,150],[325,150]])
            matrixG = cv2.getPerspectiveTransform(ptG1,ptG2)
            imgGradeDisplay = cv2.warpPerspective(img,matrixG,(325,150))

            # LOGIC => 
            # Marks Bubbles = More Pixels
            # Unmark Bubbles = Less Pixels

            # APPLY THRESHOLD
            imgWrapGray = cv2.cvtColor(imgWrapColored,cv2.COLOR_BGR2GRAY)
            imgThresh = cv2.threshold(imgWrapGray,170,255,cv2.THRESH_BINARY_INV)[1]

            # SEPARATING ALL BUBBLES
            boxes = splitBoxes(imgThresh)

            # CALCULATING PIXELS
            myPixelVal = np.zeros((5,5)) # (question,choices)
            countCol = 0
            countRow = 0

            for box in boxes:
                totalPixels = cv2.countNonZero(box)
                myPixelVal[countRow][countCol] = totalPixels
                countCol = countCol + 1

                if countCol == 5:
                    countRow = countRow + 1
                    countCol = 0

            # INDEX VALUES OF MARKING
            markingIndex = []

            for x in range(0,5):
                arr = myPixelVal[x]
                indexVal = np.where(arr == np.amax(arr))
                markingIndex.append(indexVal[0][0])


            # GRADING
            grading = []

            for x in range(0,5):
                if correctAnswers[x] == markingIndex[x]:
                    grading.append(1)
                else:
                    grading.append(0)

            score = (sum(grading) / 5) * 100

            # DISPLAYING ANSWERS
            imgRawDrawing = np.zeros_like(imgWrapColored)
            imgRawDrawing = showAnswers(imgRawDrawing,markingIndex,grading,correctAnswers)

            invMatrix = cv2.getPerspectiveTransform(pt2,pt1)
            imgInvWarp = cv2.warpPerspective(imgRawDrawing,invMatrix,(widthImg,heightImg))

            imgFinal = cv2.addWeighted(imgFinal,1,imgInvWarp,1,0)

            # DISPLAYING GRADE SCORE
            imgRawGrade = np.zeros_like(imgGradeDisplay)
            cv2.putText(imgRawGrade,str(int(score))+'%',(60,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,255),3)

            invMatrixG = cv2.getPerspectiveTransform(ptG2,ptG1)
            imgInvGradeDisplay = cv2.warpPerspective(imgRawGrade,invMatrixG,(widthImg,heightImg))

            imgFinal = cv2.addWeighted(imgFinal,1,imgInvGradeDisplay,1,0)

        imgArray = ([ imgCanny,imgContours],
           [imgThresh,imgFinal])
        imgStacked = stackImages(imgArray,0.5)
            
        cv2.imshow('Stacked',imgStacked)
        cv2.imshow('Result',imgFinal)
        
    except:
        cv2.imshow('Image',img)
        

In [27]:
cap = cv2.VideoCapture(1)

while True:
    success,img = cap.read()
    orm(img)
    
    if cv2.waitKey(1) & 0xff==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()